This notebook generates the paragraph about the microCT-scanning from logfiles of the scans.

In [ ]:
import platform
import os
import pandas
import glob

In [ ]:
from parsing_functions import *

In [ ]:
platform.system()

In [ ]:
# Different locations if running either on Linux or Windows
if 'Linux' in platform.system():
    BasePath = os.path.join(os.path.sep, 'home', 'habi', 'research_storage_uct', 'Archiv_Tape')
elif 'Windows' in platform.system():
    BasePath = os.path.join('R:', os.sep)
Root = os.path.join(BasePath, 'Fernandez Melanoma')
print('We are loading all the data from the folder %s' % Root)

In [ ]:
# Make us a dataframe for saving all that we need
Data = pandas.DataFrame()

In [ ]:
# Get *all* log files
# Using os.walk is way faster than using recursive glob.glob, see DataWrangling.ipynb for details
# Not sorting the found logfiles is also making it quicker
Data['LogFile'] = [os.path.join(root, name)
                   for root, dirs, files in os.walk(Root)
                   for name in files
                   if name.endswith((".log"))]

In [ ]:
# Get folder names
Data['Folder'] = [os.path.dirname(f) for f in Data['LogFile']]

In [ ]:
# See what we get
Data.sample(n=5)

In [ ]:
print('We found in total %s log files in %s, let us now look at them' % (len(Data), Root))

In [ ]:
# Get rid of all the logfiles from all the folders that might be on disk but that we don't want to load the data from
for c, row in Data.iterrows():
    if 'SubScan' in row.Folder:  # Prematurely synchronized folders might be present
        print('Dropping', row.LogFile, 'as we are not interested in it')
        Data.drop([c], inplace=True)
    elif 'batman' in row.LogFile: # There's nothing interesting in the Bruker batch manager log files
        Data.drop([c], inplace=True)
        print('Dropping', row.LogFile, 'as we are not interested in it')
    elif 'ctan.log' in row.LogFile: # There's nothing interesting in the Bruker CT Analzyer log files
        Data.drop([c], inplace=True)
        print('Dropping', row.LogFile, 'as we are not interested in it')
    elif 'rectmp.log' in row.LogFile: # Prematurely synchronized files might be present
        Data.drop([c], inplace=True)
        print('Dropping', row.LogFile, 'as we are not interested in it')
    elif os.path.basename(row.LogFile).startswith('._'): # Someone on macOS might have looked at the files
        Data.drop([c], inplace=True)
        print('Dropping', row.LogFile, 'as we are not interested in it')        
# Reset dataframe to something that we would get if we only would have loaded the 'rec' files
Data = Data.reset_index(drop=True)

In [ ]:
print('After cleanup, we now have a total of %s log files to look at' % len(Data))

In [ ]:
Data['Scanner'] = [scanner(log) for log in Data['LogFile']]
Data['Software'] = [controlsoftware(log) for log in Data['LogFile']]

In [ ]:
Data['Voxelsize'] = [pixelsize(log) for log in Data['LogFile']]
Data['Voxelsize_rounded'] = [pixelsize(log,rounded=True) for log in Data['LogFile']]

In [ ]:
Data['Source'] = [source(log) for log in Data['LogFile']]
Data['Camera'] = [camera(log) for log in Data['LogFile']]
Data['Exposure'] = [exposuretime(log) for log in Data['LogFile']]
Data['Averaging'] = [averaging(log) for log in Data['LogFile']]

In [ ]:
Data['Voltage'] = [voltage(log) for log in Data['LogFile']]
Data['Current'] = [current(log) for log in Data['LogFile']]
Data['Filter'] = [whichfilter(log) for log in Data['LogFile']]

In [ ]:
Data['Stacks'] = [stacks(log) for log in Data['LogFile']]
Data['NumProj'] = [numproj(log) for log in Data['LogFile']]
Data['ProjSize'] = [projection_size(log) for log in Data['LogFile']]
Data['RotationStep'] = [rotationstep(log) for log in Data['LogFile']]
Data['Wide'] = [overlapscan(log) for log in Data.LogFile]

In [ ]:
Data['Version'] = [nreconversion(log) for log in Data['LogFile']]
Data['RingRemoval'] = [ringremoval(log) for log in Data['LogFile']]
Data['Beamhardening'] = [beamhardening(log) for log in Data['LogFile']]
Data['DefectPixelMasking'] = [defectpixelmasking(log) for log in Data['LogFile']]
Data['GrayValue'] = [reconstruction_grayvalue(log) for log in Data['LogFile']]

In [ ]:
Data['RecSize'] = [reconstruction_size(log) for log in Data['LogFile']]
Data['ROI'] = [region_of_interest(log, verbose=False) for log in Data['LogFile']]
Data['RecRotation'] = [crosssection_rotation(log, verbose=False) for log in Data['LogFile']]

In [ ]:
Data['Duration'] = [duration(log) for log in Data['LogFile']]
Data['Date'] = [scandate(log) for log in Data['LogFile']]

In [ ]:
# Sort data by scan date
Data.sort_values(by='Date', inplace=True, ignore_index=True)

In [ ]:
Data.to_csv('ScanningDetails.csv')

----

My microct blurb from http://simp.ly/publish/NBhZhH

Cris mentioned that he used the scans "D [...] for [...] Figure 2".
Let's make us a subset of the dataframe for adding the text into the manuscript.

In [ ]:
# Only look at the scans of the 'D' samples
Data_Figure2 = Data[Data['LogFile'].str.contains('D')]
# Get rid of the irrelevant sample "646L_Dry"
Data_Figure2 = Data_Figure2.drop(Data_Figure2[Data_Figure2['LogFile'].str.contains('646L')].index)
# Drop all *rec* log files
Data_Figure2 = Data_Figure2.drop(Data_Figure2[Data_Figure2['LogFile'].str.contains('rec')].index)
# Drop all subscan log files
Data_Figure2 = Data_Figure2.drop(Data_Figure2[Data_Figure2['LogFile'].str.contains('~')].index)
# Drop the four scans done on the 2214, which were not used for Figure 2
Data_Figure2 = Data_Figure2.drop(Data_Figure2[Data_Figure2['Scanner'].str.contains('2214')].index)
# Drop one scan that was scanned 'too_high'
Data_Figure2 = Data_Figure2.drop(Data_Figure2[Data_Figure2['LogFile'].str.contains('too')].index)

In [ ]:
print('Based on the %s log files containing a "D" in their name' % len(Data_Figure2))

In [ ]:
# What do we have now?
Data_Figure2.LogFile.unique()

In [ ]:
print('After $PREPARATION, the',
      len(Data_Figure2),
      'samples were imaged on a Bruker',
      " OR ".join(str(value) for value in Data_Figure2.Scanner.unique()),
      'high-resolution microtomography machine (Control software version',
      " OR ".join(str(value) for value in Data_Figure2.Software.unique()) + 
      ', Bruker microCT, Kontich, Belgium).')

In [ ]:
print('The machine is equipped with a',
      " OR ".join(str(value) for value in Data_Figure2.Source.unique()),
      'X-ray source and a',
      " OR ".join(str(value) for value in Data_Figure2.Camera.unique()),
      'camera.')

In [ ]:
print('The X-ray source was set to a tube voltage of', 
      " OR ".join(str(value) for value in Data_Figure2.Voltage.unique()),
      'kV and a tube current of',
      " OR ".join(str(value) for value in Data_Figure2.Current.unique()),
      'µA, the x-ray spectrum was', end=' ')
if Data_Figure2.Filter.unique():
    print('filtered by', " OR ".join(str(value) for value in Data_Figure2.Filter.unique()), end=' ')
else:
    print('not filtered', end=' ')
print('prior to incidence onto the sample.')

In [ ]:
print('For each sample, we recorded a set of', end=' ')
if Data_Figure2.Filter.unique():   
    print(" or ".join(str(value) for value in Data_Figure2.Stacks.unique()),
          'stacked scans overlapping the sample height, each stack was recorded with', end=' ')
print(" or ".join(str(value) for value in Data_Figure2.NumProj.unique()), 'projections with a size of', end=' ')
for cs in Data_Figure2.ProjSize.unique():
    print(cs[0], end=' ')
print('x', end=' ')
for cs in Data_Figure2.ProjSize.unique():
    print(cs[1], end=' ')
print('pixels each', end=' ')
if Data_Figure2.Wide.unique():
    print('(' + " or ".join(str(value) for value in Data_Figure2.Wide.unique()), 'projections stitched laterally)', end=' ')
print('at every',
      str(" or ".join(str(value) for value in Data_Figure2.RotationStep.unique())) + '° over a 180° sample rotation.')

In [ ]:
Data_Figure2.Exposure.mean()

In [ ]:
print('Every single projection was exposed for',
      " or ".join(str(value) for value in Data_Figure2.Exposure.unique()),
      'ms,',
      " or ".join(str(value) for value in Data_Figure2.Averaging.unique()),
      'projections were averaged to one to greatly reduce image noise.')

In [ ]:
Data_Figure2.Duration.mean()

In [ ]:
def timeformat(tdelta, fmt):
    # From https://stackoverflow.com/a/8907269/323100
    d = {"days": tdelta.days}
    d["hours"], rem = divmod(tdelta.seconds, 3600)
    d["minutes"], d["seconds"] = divmod(rem, 60)
    return fmt.format(**d)

In [ ]:
print('This resulted in a scan time of approximately ', end='')
print(timeformat(datetime.timedelta(seconds=Data_Figure2.Duration.mean()),
                     '{hours} hours and {minutes} minutes'), end=' ')
print('per sample (with %s stack per sample))' % Data_Figure2.Stacks.unique())

In [ ]:
print('In total, we scanned', Data_Figure2.Stacks.sum(), 'stacks.')
print('Each stack took approximately',
      Data_Figure2.Duration.mean() // 60,
      'minutes (' + str(datetime.timedelta(seconds=Data_Figure2.Duration.mean())) + ')')
print('In total, we thus scanned for about', 
      timeformat(Data_Figure2.Stacks.sum() *
                 datetime.timedelta(seconds=Data_Figure2.Duration.mean()),
                 '{days} days, {hours} hours and {minutes} minutes.'))
print('At the MIC rate, this would have cost',
      int(round(Data_Figure2.Stacks.sum() * Data_Figure2.Duration.mean() / 60 / 60 * 75)),
      'CHF.')

In [ ]:
print('The projection images were then subsequently reconstructed into a 3D stack',
      'of images with',
      Data_Figure2.Version.unique()[0][0],
      '(Version',
      nreconversion(log)[1] + ', Bruker microCT, Kontich Belgium)', end=' ')
if ringremoval(log):
    print('using a ring artifact correction of',
          ringremoval(log), end='')
if beamhardening(log):
    print(' and a beam hardening correction of',
          beamhardening(log),
          '%.')
else:
    print('.')
print('The whole process resulted in datasets with an isometric voxel size of',
      " or ".join(str(value) for value in Data_Figure2.Voxelsize_rounded.unique()),
      'µm.')    

In [ ]:
# fulllog(log)

In [ ]:
Data_Figure2.Voxelsize.mean()

In [ ]:
Data_Figure2.Beamhardening